In [6]:
import optuna
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load the local dataset
file_paths = [
    "C:/VS code projects/data_files/Monday-WorkingHours.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Tuesday-WorkingHours.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Wednesday-workingHours.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Friday-WorkingHours-Morning.pcap_ISCX.csv"
]



# Read and clean datasets
dataframes = []
for file_path in file_paths:
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()  # Remove whitespace from column names
    dataframes.append(df)

# Combine all datasets into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)
df['Label'] = df['Label'].apply(lambda x: 1 if x == 'BENIGN' else 0)


In [7]:

# Find the minimum class count
min_count = df['Label'].value_counts().min()


# Perform undersampling to balance the dataset
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42, ignore_index=True))

# Preprocessing dataset
print("Preprocessing dataset...")
df.dropna(inplace=True)  # Remove missing values
df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infinite values with NaN
df.columns = df.columns.str.strip()  # Strip whitespaces from column names

# Separate features and labels
X = df.drop('Label', axis=1)  # Features
Y = df['Label']              # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    param_grid = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
    }

    # Train LightGBM with the current set of hyperparameters
    model = lgb.LGBMClassifier(**param_grid, random_state=42)  # Unpack param_grid using **
    model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_test, y_test)], 
        eval_metric='logloss', 
        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)]  # Use callbacks for verbosity and early stopping
    )

    # Predict on the validation set
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Return the accuracy as the objective to maximize
    return accuracy

# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')  # Maximize accuracy
study.optimize(objective, n_trials=50, timeout=3600)  # Run 50 trials or stop after 1 hour

# Print the best hyperparameters
print("Best Hyperparameters:")
# Saving the hyperparameters to a file in the "data" folder
output_file = "models_and_data/best_hyperparametersLightGBM.txt"
with open(output_file, "w") as file:
    file.write("Best Hyperparameters:\n")
    for key, value in study.best_params.items():
        file.write(f"{key}: {value}\n")
    file.write(f"\nBest Score: {study.best_value}\n")

print(f"Best hyperparameters and score saved to {output_file}")


C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42, ignore_index=True))


Preprocessing dataset...


[I 2025-04-13 07:00:00,971] A new study created in memory with name: no-name-791e5347-975d-4e02-8ece-1547ff0908d3
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:31: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/o

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 390213, number of negative: 389764
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.468310 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14459
[LightGBM] [Info] Number of data points in the train set: 779977, number of used features: 70
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500288 -> initscore=0.001151
[LightGBM] [Info] Start training from score 0.001151
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[137]	valid_0's binary_logloss: 0.00316565


[I 2025-04-13 07:00:35,305] Trial 0 finished with value: 0.9994166497146071 and parameters: {'learning_rate': 0.05274355057186725, 'num_leaves': 113, 'max_depth': 12, 'min_child_samples': 39, 'subsample': 0.8923986695695098, 'colsample_bytree': 0.5881835435207168, 'reg_alpha': 0.0012627853480256424, 'reg_lambda': 2.0579678685195358e-05, 'n_estimators': 137}. Best is trial 0 with value: 0.9994166497146071.
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 390213, number of negative: 389764
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.344120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14455
[LightGBM] [Info] Number of data points in the train set: 779977, number of used features: 68
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500288 -> initscore=0.001151
[LightGBM] [Info] Start training from score 0.001151
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2025-04-13 07:21:13,274] Trial 48 finished with value: 0.9994196412545322 and parameters: {'learning_rate': 0.24641025177084844, 'num_leaves': 61, 'max_depth': 9, 'min_child_samples': 76, 'subsample': 0.8220464397199156, 'colsample_bytree': 0.6662803279195477, 'reg_alpha': 0.3403048245468236, 'reg_lambda': 0.022339844591600273, 'n_estimators': 335}. Best is trial 47 with value: 0.9994495566537831.
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_21700\3830879630.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Us

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 390213, number of negative: 389764
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.301444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14455
[LightGBM] [Info] Number of data points in the train set: 779977, number of used features: 68
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500288 -> initscore=0.001151
[LightGBM] [Info] Start training from score 0.001151
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2025-04-13 07:21:26,334] Trial 49 finished with value: 0.999413658174682 and parameters: {'learning_rate': 0.21193410794748088, 'num_leaves': 72, 'max_depth': 9, 'min_child_samples': 67, 'subsample': 0.8069268232212223, 'colsample_bytree': 0.6046528595341005, 'reg_alpha': 1.1208865741851617, 'reg_lambda': 0.002015115672065376, 'n_estimators': 304}. Best is trial 47 with value: 0.9994495566537831.


Best Hyperparameters:
Best hyperparameters and score saved to models_and_data/best_hyperparametersLightGBM.txt
